In [1]:
%cd ..

/Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu 18th night


# Bayesian optimization: finding the minimum time loss

In this notebook we will performing bayesian optimization on the emulator trained on our sumo simulation. We are interested in identifying the parameters which enable us to obtain the minimum travel time. Our emulator is a gaussian process with an **rbf** kernel.


## Imports

In [2]:
import pickle
import numpy as np
import emukit as ek
import GPy
import pandas as pd

from emukit.model_wrappers import GPyModelWrapper
from emukit.core.initial_designs import RandomDesign
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop

from sumo_grid_simulation.grid_simulation import Simulator
from experimental_design import config

## Simulator

We start by initialising our simulation.

In [3]:
simulator = Simulator(end_time=300)

We then define the two user functions in which we are interested:

In [4]:
def user_function_time_loss(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function time loss called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        beta = 0.05
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * beta)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        # average time loss / average route duration
        result.append(s['timeLoss']/s['duration'])
        print(f'\nOutput {result[-1]}\n')
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

We then load the parameters' space from our configuration file:

In [5]:
parameter_space = config.get_parameter_space()

## Emulator

### Rnadomly initialised emulator

The first emulator we are going to analyse is the one that is just initilised with random points

We first sample at random 200 datapoints from the parameter space:

In [ ]:
#init_X, init_Y = pickle.load(open('bayesian_optimization/init_points/520_init_points_timeLoss.pkl', 'rb'))

'''
design = RandomDesign(parameter_space)
num_init_points = 520
init_X = design.get_samples(num_init_points)
init_Y = user_function_time_loss(init_X)
init_points = init_X, init_Y
with open(f'bayesian_optimization/init_points/{num_init_points}_init_points_timeLoss.pkl', "wb") as f:
     pickle.dump(init_points, f)
'''

[[20.         11.36676831 30.57219306  2.          2.4945695 ]
 [ 5.          8.72593315 38.43436756  2.          2.12985565]
 [18.         12.72195183 53.00240726  3.          4.75483272]
 ...
 [ 8.         16.7749665  33.76908728  1.          3.77541186]
 [ 8.         10.72705993 35.72697109  2.          1.6557556 ]
 [16.         16.68808991 50.68652082  2.          2.06663221]]

User function time loss called with 520 inputs to simulate

Evaluating input: 1 of 520

 Retrying in 1 seconds

Output 0.27952690321505913


Evaluating input: 2 of 520

 Retrying in 1 seconds

Output 0.19130150425114453


Evaluating input: 3 of 520

 Retrying in 1 seconds

Output 0.2722274216190922


Evaluating input: 4 of 520

 Retrying in 1 seconds

Output 0.2447104082068818


Evaluating input: 5 of 520

 Retrying in 1 seconds

Output 0.37267625394598386


Evaluating input: 6 of 520

 Retrying in 1 seconds

Output 0.35581245985870263


Evaluating input: 7 of 520

 Retrying in 1 seconds

Output 0.3948142699

 Retrying in 1 seconds

Output 0.30994364660970736


Evaluating input: 97 of 520

 Retrying in 1 seconds

Output 0.3550139963774082


Evaluating input: 98 of 520

 Retrying in 1 seconds

Output 0.2972222222222222


Evaluating input: 99 of 520

 Retrying in 1 seconds

Output 0.36053882725832015


Evaluating input: 100 of 520

 Retrying in 1 seconds

Output 0.25811437403400306


Evaluating input: 101 of 520

 Retrying in 1 seconds

Output 0.4408751038493492


Evaluating input: 102 of 520

 Retrying in 1 seconds

Output 0.4545722713864307


Evaluating input: 103 of 520

 Retrying in 1 seconds

Output 0.404


Evaluating input: 104 of 520

 Retrying in 1 seconds

Output 0.18309614717112116


Evaluating input: 105 of 520

 Retrying in 1 seconds

Output 0.31571399303421427


Evaluating input: 106 of 520

 Retrying in 1 seconds

Output 0.544154540893126


Evaluating input: 107 of 520

 Retrying in 1 seconds

Output 0.24092476171162036


Evaluating input: 108 of 520

 Retrying in 1 seconds

Out

 Retrying in 1 seconds

Output 0.4060795011691348


Evaluating input: 197 of 520

 Retrying in 1 seconds

Output 0.20585906571654788


Evaluating input: 198 of 520

 Retrying in 1 seconds

Output 0.39847133757961783


Evaluating input: 199 of 520

 Retrying in 1 seconds

Output 0.3519359723123513


Evaluating input: 200 of 520

 Retrying in 1 seconds

Output 0.3638149939540508


Evaluating input: 201 of 520

 Retrying in 1 seconds

Output 0.39314456035767514


Evaluating input: 202 of 520

 Retrying in 1 seconds

Output 0.33958333333333335


Evaluating input: 203 of 520

 Retrying in 1 seconds

Output 0.33383010432190763


Evaluating input: 204 of 520

 Retrying in 1 seconds

Output 0.2353960648384137


Evaluating input: 205 of 520

 Retrying in 1 seconds

Output 0.33651172030194676


Evaluating input: 206 of 520

 Retrying in 1 seconds

Output 0.19570528481419572


Evaluating input: 207 of 520

 Retrying in 1 seconds

Output 0.26771237222418043


Evaluating input: 208 of 520

 Retryin

 Retrying in 1 seconds

Output 0.2640491057254736


Evaluating input: 297 of 520

 Retrying in 1 seconds

Output 0.3137490007993605


Evaluating input: 298 of 520

 Retrying in 1 seconds

Output 0.16854201580081396


Evaluating input: 299 of 520

 Retrying in 1 seconds

Output 0.28104369695064446


Evaluating input: 300 of 520

 Retrying in 1 seconds

Output 0.26370757180156656


Evaluating input: 301 of 520

 Retrying in 1 seconds

Output 0.39098011363636365


Evaluating input: 302 of 520

 Retrying in 1 seconds

Output 0.394801545486477


Evaluating input: 303 of 520

 Retrying in 1 seconds

Output 0.36


Evaluating input: 304 of 520

 Retrying in 1 seconds

Output 0.2728068895051817


Evaluating input: 305 of 520

 Retrying in 1 seconds

Output 0.3107951247823564


Evaluating input: 306 of 520

 Retrying in 1 seconds

Output 0.29227124666864074


Evaluating input: 307 of 520

 Retrying in 1 seconds

Output 0.211963589076723


Evaluating input: 308 of 520

 Retrying in 1 seconds

Out

 Retrying in 1 seconds

Output 0.18277852843631004


Evaluating input: 397 of 520

 Retrying in 1 seconds

Output 0.42567567567567566


Evaluating input: 398 of 520

 Retrying in 1 seconds

Output 0.3236794924837953


Evaluating input: 399 of 520

 Retrying in 1 seconds

Output 0.41718838929994306


Evaluating input: 400 of 520

 Retrying in 1 seconds

Output 0.22191500256016386


Evaluating input: 401 of 520

 Retrying in 1 seconds

Output 0.24823997551270274


Evaluating input: 402 of 520

 Retrying in 1 seconds

Output 0.21275469700979094


Evaluating input: 403 of 520

 Retrying in 1 seconds

Output 0.42388953585093997


Evaluating input: 404 of 520

 Retrying in 1 seconds

Output 0.1827637444279346


Evaluating input: 405 of 520

 Retrying in 1 seconds

Output 0.29110666285387476


Evaluating input: 406 of 520

 Retrying in 1 seconds

Output 0.35177182368193605


Evaluating input: 407 of 520

 Retrying in 1 seconds

Output 0.25183268650280294


Evaluating input: 408 of 520

 Retry

To then fit a GP to these points

In [ ]:
#emulator = GPy.models.GPRegression(init_X, init_Y, noise_var=1e-10)
#emukit_model_random_init = GPyModelWrapper(emulator, n_restarts=5)
#emukit_model_random_init.optimize()

### Experimentally designed emulator with model variance

We also load the emulator obtained during experimental design:

In [ ]:
#emukit_model_variance = pickle.load(open('experimental_design/models/5param_model_variance_20_init_points_500_loops_timeloss_per_duration.pkl', 'rb'))

### Experimentally designed emulator with integrated variance reduction

We also load the emulator obtained during experimental design:

In [6]:
emukit_model_integrated_variance = pickle.load(open('experimental_design/models/5param_integrated_variance_reduction_20_init_points_500_loops_timeloss_per_duration.pkl', 'rb'))

## Bayesian optimisation

We now run bayesian optimisation on the emulators

In [7]:
n_iter_bo = 400

### Acquisition functions

In [ ]:
#acquisition_random_init = ExpectedImprovement(emukit_model_random_init)

In [ ]:
#acquisition_model_variance = ExpectedImprovement(emukit_model_variance)

In [8]:
acquisition_integrated_variance = ExpectedImprovement(emukit_model_integrated_variance)

### Optimisation loop

In [ ]:
#bo_random_init = BayesianOptimizationLoop(parameter_space, emukit_model_random_init, acquisition=acquisition_random_init)
#bo_random_init.run_loop(user_function_time_loss, n_iter_bo)

In [ ]:
#bo_model_variance = BayesianOptimizationLoop(parameter_space, emukit_model_variance, acquisition=acquisition_model_variance)
#bo_model_variance.run_loop(user_function_time_loss, n_iter_bo)

In [9]:
bo_integrated_variance = BayesianOptimizationLoop(parameter_space, emukit_model_integrated_variance, acquisition=acquisition_integrated_variance)
bo_integrated_variance.run_loop(user_function_time_loss, n_iter_bo)

Optimization restart 1/5, f = -1371.5809591212437
Optimization restart 2/5, f = -1371.5809591040568
Optimization restart 3/5, f = -1371.5809591241193
Optimization restart 4/5, f = -1371.5809591252932
Optimization restart 5/5, f = -1371.5809588287018
[[20.   8.  70.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.13209992929530992

Optimization restart 1/5, f = -1374.6873220993268
Optimization restart 2/5, f = -1374.687370405387
Optimization restart 3/5, f = -1374.6873703912938
Optimization restart 4/5, f = -1374.6873703878896
Optimization restart 5/5, f = -1374.6873703997949
[[10.          8.         70.          2.          1.87547802]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.12663661729984976

Optimization restart 1/5, f = -1377.993074829421
Optimization restart 2/5, f = -1377.9931594839923
Optimization restart 3/5, f = -1377

 Retrying in 1 seconds

Output 0.13258376934209062

Optimization restart 1/5, f = -1436.6374199296265
Optimization restart 2/5, f = -1436.6374571362571
Optimization restart 3/5, f = -1436.6374572572731
Optimization restart 4/5, f = -1436.6374572598297
Optimization restart 5/5, f = -1436.6374572620384
[[10.          8.         70.          2.          1.86594621]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.1298770710849813

Optimization restart 1/5, f = -1440.1615692847147
Optimization restart 2/5, f = -1440.161602456781
Optimization restart 3/5, f = -1440.1616022868295
Optimization restart 4/5, f = -1440.161602461013
Optimization restart 5/5, f = -1440.1616016124663
[[11.          8.         62.14301774  1.          1.947226  ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.1329252295583734

Optimization restart 1/5, f = -1443.6683148583138


Optimization restart 5/5, f = -1499.83072643395
[[20.          8.         62.049175    1.          1.57743864]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.13823038397328882

Optimization restart 1/5, f = -1503.3047675849018
Optimization restart 2/5, f = -1503.304798136978
Optimization restart 3/5, f = -1503.304798641363
Optimization restart 4/5, f = -1503.3047986164224
Optimization restart 5/5, f = -1503.3047986669371
[[10.          8.         70.          2.          1.84128729]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.12852093820284888

Optimization restart 1/5, f = -1506.8660966441128
Optimization restart 2/5, f = -1506.8661377017065
Optimization restart 3/5, f = -1506.8661380368399
Optimization restart 4/5, f = -1506.8661379932896
Optimization restart 5/5, f = -1506.8661380323504
[[10.          8.         68.16677747  1.          1

 Retrying in 1 seconds

Output 0.12640841291984117

Optimization restart 1/5, f = -1566.9523711663176
Optimization restart 2/5, f = -1566.952411817553
Optimization restart 3/5, f = -1566.9524118184936
Optimization restart 4/5, f = -1566.9524118360196
Optimization restart 5/5, f = -1566.9524118207885
[[10.          8.         70.          2.          1.81067196]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.12640841291984117

Optimization restart 1/5, f = -1570.5333495172442
Optimization restart 2/5, f = -1570.5333902751047
Optimization restart 3/5, f = -1570.5333902571379
Optimization restart 4/5, f = -1570.5333902790712
Optimization restart 5/5, f = -1570.5333902928544
[[20.          8.         61.61742057  1.          1.58801992]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.13650110894549364

Optimization restart 1/5, f = -1574.08387965125

Optimization restart 3/5, f = -1631.4903228178232
Optimization restart 4/5, f = -1631.4903228134424
Optimization restart 5/5, f = -1631.4903226975562
[[10.          8.         70.          2.          1.78171969]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.12599270229663018

Optimization restart 1/5, f = -1635.1005610490301
Optimization restart 2/5, f = -1635.1006017643626
Optimization restart 3/5, f = -1635.1006038989153
Optimization restart 4/5, f = -1635.1006038973676
Optimization restart 5/5, f = -1635.1005385616627
[[10.          8.         70.          2.          1.78151031]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.12599270229663018

Optimization restart 1/5, f = -1638.712639251406
Optimization restart 2/5, f = -1638.7126824920367
Optimization restart 3/5, f = -1638.7126825020682
Optimization restart 4/5, f = -1638.7126825244763

...

[[12.          8.         70.          1.          1.51685041]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.12400256810052279

Optimization restart 1/5, f = -2521.9744322112083
Optimization restart 2/5, f = -2465.6936827258355
Optimization restart 3/5, f = -2521.974513863614
Optimization restart 4/5, f = -2465.693682755062
Optimization restart 5/5, f = -2465.6936827280433
[[12.          8.         70.          1.          1.51106524]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.12411770098084149

Optimization restart 1/5, f = -2525.7919123540178
Optimization restart 2/5, f = -2470.8463256824552
Optimization restart 3/5, f = -2470.8463256734
Optimization restart 4/5, f = -2470.846325682348
Optimization restart 5/5, f = -2470.8463256317236
[[12.          8.         70.          1.          1.50629659]]

User function time loss called with 1 i

Optimization restart 2/5, f = -2564.589975753674
Optimization restart 3/5, f = -2564.5899757279367
Optimization restart 4/5, f = -2564.5899757590787
Optimization restart 5/5, f = -2564.589975758878
[[12.   8.  70.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.1250801208680524

Optimization restart 1/5, f = -2598.7891715896963
Optimization restart 2/5, f = -2569.864468596987
Optimization restart 3/5, f = -2569.8644690170718
Optimization restart 4/5, f = -2569.8644691228274
Optimization restart 5/5, f = -2598.7893811103622
[[12.   8.  70.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.1250801208680524

Optimization restart 1/5, f = -2602.6435345738983
Optimization restart 2/5, f = -2575.1430518123266
Optimization restart 3/5, f = -2575.1430518187713
Optimization restart 4/5, f = -2575.1430518184216
Optimization restart 5/


Output 0.1250801208680524

Optimization restart 1/5, f = -2670.128301524163
Optimization restart 2/5, f = -2667.516768556738
Optimization restart 3/5, f = -2667.5167686094355
Optimization restart 4/5, f = -2667.516768612226
Optimization restart 5/5, f = -2667.516768556632
[[12.   8.  70.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.1250801208680524

Optimization restart 1/5, f = -2673.996283551127
Optimization restart 2/5, f = -2672.855820191702
Optimization restart 3/5, f = -2672.855820191435
Optimization restart 4/5, f = -2672.855820191109
Optimization restart 5/5, f = -2672.8558201826427
[[12.   8.  70.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.1250801208680524

Optimization restart 1/5, f = -2677.864761691844
Optimization restart 2/5, f = -2678.197834879326
Optimization restart 3/5, f = -2678.197834864526
Opt


Output 0.15651006711409396

Optimization restart 1/5, f = -2713.9342114890565
Optimization restart 2/5, f = -2713.93421150027
Optimization restart 3/5, f = -2713.9342114552337
Optimization restart 4/5, f = -2713.9342113961843
Optimization restart 5/5, f = -2713.934211505588
[[ 3.          8.         45.24506721  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.19437432383699962

Optimization restart 1/5, f = -2716.0543561914465
Optimization restart 2/5, f = -2716.0543561939385
Optimization restart 3/5, f = -2716.0543561893437
Optimization restart 4/5, f = -2716.0543559232538
Optimization restart 5/5, f = -2716.0543546678064
[[ 3.          8.         55.74923949  1.          1.56960976]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.1622729935559461

Optimization restart 1/5, f = -2718.3124142008382
Optimization restart 2/

 Retrying in 1 seconds

Output 0.1250801208680524

Optimization restart 1/5, f = -2810.1018905569063
Optimization restart 2/5, f = -2784.975566782062
Optimization restart 3/5, f = -2784.9755667857744
Optimization restart 4/5, f = -2784.9755667881177
Optimization restart 5/5, f = -2784.9755662111247
[[12.   8.  70.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.1250801208680524

Optimization restart 1/5, f = -2813.992455604437
Optimization restart 2/5, f = -2790.343198083947
Optimization restart 3/5, f = -2790.3431980818978
Optimization restart 4/5, f = -2790.3431980850146
Optimization restart 5/5, f = -2790.3431980713913
[[12.   8.  70.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.1250801208680524

Optimization restart 1/5, f = -2817.8837602855424
Optimization restart 2/5, f = -2795.7134958370334
Optimization restart 3

## Results

In [10]:
parameters_name = ['Size of the grid', 'Speed limit in the net (m/s)', 'Length of the roads (m)', 'Number of lanes', 'Accelleration (m/s^2)']

### Results on the random initialised emulator

In [ ]:
#results_random_init = bo_random_init.get_results()

In [ ]:
#results_random_init_df = pd.DataFrame(results_random_init.minimum_location.reshape(1,-1))
#results_random_init_df.columns = parameters_name
#results_random_init_df

In [ ]:
#results_random_init.minimum_value*100

### Results on the experimentally designed emulator with model variance

In [ ]:
#results_model_variance = bo_model_variance.get_results()

In [ ]:
#results_random_init_df = pd.DataFrame(results_model_variance.minimum_location.reshape(1,-1))
#results_random_init_df.columns = parameters_name
#results_random_init_df

In [ ]:
#results_model_variance.minimum_value*100

### Results on the experimentally designed emulator with integrated variance reduction

In [11]:
results_integrated_variance = bo_integrated_variance.get_results()

In [12]:
results_random_init_df = pd.DataFrame(results_integrated_variance.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,12.0,8.0,70.0,1.0,1.567259


In [13]:
results_integrated_variance.minimum_value*100

12.26345765203013

## Save stuff

In [14]:
# with open(f'bayesian_optimization/bayesian_opt_results/results_random_init_timeLoss.pkl', "wb") as f:
#      pickle.dump(results_random_init, f)

# with open(f'bayesian_optimization/bayesian_opt_results/results_model_variance_timeLoss.pkl', "wb") as f:
#      pickle.dump(results_model_variance, f)
        
with open(f'bayesian_optimization/bayesian_opt_results/results_integrated_variance_timeLoss.pkl', "wb") as f:
     pickle.dump(results_integrated_variance, f)

In [15]:
# with open(f'bayesian_optimization/models/520_random_init_50_bay_opt_timeLoss.pkl', "wb") as f:
#      pickle.dump(emukit_model_random_init, f)

# with open(f'bayesian_optimization/models/520_model_variance_50_bay_timeLoss.pkl', "wb") as f:
#      pickle.dump(emukit_model_variance, f)
        
with open(f'bayesian_optimization/models/520_integrated_variance_50_bay_timeLoss', "wb") as f:
     pickle.dump(emukit_model_integrated_variance, f)